In [75]:
import tensorflow as tf
import keras
import numpy as np
import os
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
import seaborn as sns

import scipy.stats as sc_stats

import random

from sklearn.model_selection import train_test_split
onehot_encoder=OneHotEncoder(sparse=False)

In [164]:
# parameters
time_steps = 1
inputs = 3
outputs = 2

In [165]:
def add_releveant_features(task_df):


    task_df['PrevOutcome']=task_df['Outcome'].shift(1)
    task_df.loc[1,'PrevOutcome']= 0

    task_df['PrevChoice']=task_df['Choice'].shift(1)
    task_df.loc[1,'PrevChoice']= 0

    task_df['PrevSafe']=task_df['Safe'].shift(1)
    task_df.loc[1,'PrevSafe']= 0

    task_df['PrevBigRisky']=task_df['BigRisky'].shift(1)
    task_df.loc[1,'PrevBigRisky']= 0

    task_df['PrevSmallRisky']=task_df['SmallRisky'].shift(1)
    task_df.loc[1,'PrevSmallRisky']= 0
    
#     task_df['PrevRT']=task_df['RT'].shift(1)
#     task_df.loc[1,'PrevRT']= 0
    
    
    
    return task_df

In [166]:
def chunk_split_data(data,start_chunk,end_chunk):
    
    a=[k for k in range(start_chunk,end_chunk)]
    out=[]

    for d in range(0,data.shape[0],20):

        c= [c+d for c in a]
        out = out+c

    while out[-1]>=data.shape[0]-1:
        out.pop()
#     return out
    return data[out]

In [167]:
def train_RNN(neurons,train_X,train_y,test_X,test_y,val_X,val_y): 
    reset_graph()

    learning_rate = 0.001
    epochs = 50000
    batch_size = int(train_X.shape[0]/2)
    # batch_size = 100
    length = train_X.shape[0]
    display = 100
    neurons = neurons

    num_batches = 100
    seq_len = 10

    percent_above_PT = 1

    train_threshold = 1.5#PT_R2 + percent_above_PT


    save_step = 100


    best_loss_val = np.infty
    checks_since_last_progress = 0
    max_checks_without_progress = 1000


    # clear graph (if any) before running
    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, [None, time_steps, inputs])

    y = tf.placeholder(tf.float32, [None, outputs])

    # LSTM Cell
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=neurons, activation=tf.nn.relu)
    cell_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

    # pass into Dense layer
    stacked_outputs = tf.reshape(cell_outputs, [-1, neurons])
    out = tf.layers.dense(inputs=stacked_outputs, units=outputs)

    probability = tf.nn.softmax(out)

    # squared error loss or cost function for linear regression
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=y, logits=out))

    # optimizer to minimize cost
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    saver = tf.train.Saver()
    
    accuracy = tf.metrics.accuracy(labels =  tf.argmax(y, 1),
                          predictions = tf.argmax(out, 1),
                          name = "accuracy")
    precision = tf.metrics.precision(labels=tf.argmax(y, 1),
                                 predictions=tf.argmax(out, 1),
                                 name="precision")
    recall = tf.metrics.recall(labels=tf.argmax(y, 1),
                           predictions=tf.argmax(out, 1),
                           name="recall")
    f1 = 2 * accuracy[1] * recall[1] / ( precision[1] + recall[1] )

    acc_up,acc_val = accuracy
    auc = tf.metrics.auc(labels=tf.argmax(y, 1),
                           predictions=tf.argmax(out, 1),
                           name="auc")
    
    valid_store = []
    
    with tf.Session() as sess:
        #######################
#         saver.restore(sess, "./checkpts/Original_RNN_LSTM_8features_v2.ckpt")
#         saver.restore(sess, "./checkpts/OriginalDATA_RNN_LSTM_8features.ckpt")
        
        if pretraining == True:

            saver.restore(sess, "./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt")

        #######################
        
        # initialize all variables
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        # Train the model
        for steps in range(epochs):
            mini_batch = zip(range(0, length, batch_size),
                       range(batch_size, length+1, batch_size))

            # train data in mini-batches
            for (start, end) in mini_batch:
    #             print(start,end)
                sess.run(training_op, feed_dict = {X: train_X[start:end,:,:],
                                                   y: train_y[start:end,:]}) 

            ## train data in batches of length subsequence

    #         for k in range(num_batches):
    #             X_seq, y_seq = random_subsequence(train_X,train_y,seq_len)

    #             sess.run(training_op, feed_dict = {X:X_seq,y:y_seq}) 
            loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
            loss_val = loss.eval(feed_dict={X: val_X, y: val_y})


            # print training performance 
            if (steps+1) % display == 0:
                # evaluate loss function on training set


                loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
                print('Step: {}  \tTraining loss: {}'.format((steps+1), loss_fn))

                acc_train = acc_val.eval(feed_dict={X: train_X, y: train_y})
                print('Step: {}  \tTraining accuracy: {}'.format((steps+1), acc_train))


                acc_test = acc_val.eval(feed_dict={X: test_X, y: test_y})
    #             print('Step: {}  \tTest accuracy: {}'.format((steps+1), acc_test))

                loss_test = loss.eval(feed_dict={X: test_X, y: test_y})
    #             print('Step: {}  \tTest loss: {}'.format((steps+1), loss_test))

                accu_val = acc_val.eval(feed_dict={X: val_X, y: val_y})

                loss_val = loss.eval(feed_dict={X: val_X, y: val_y})
                print('Step: {}  \tValid loss: {}'.format((steps+1), loss_val))

                valid_store.append(loss_val)







            if (1 + loss_fn/np.log(0.5)) > train_threshold:
                    print("Threshold achieved, quit training")
                    break


            if loss_val < best_loss_val:

                        best_loss_val = loss_val
                        checks_since_last_progress = 0
            else:
                            checks_since_last_progress += 1


            # EARLY STOPPING
            if checks_since_last_progress > max_checks_without_progress:
                print("Early stopping!")
                break


            if (steps+1) % save_step ==0:
                                save_path = saver.save(sess, "./checkpts/Later_RNN_LSTM_8features.ckpt")

#                 save_path = saver.save(sess, "./checkpts/RNN_Internet_LSTM_model_5features.ckpt")





    #     evaluate model accuracy
        acc, prec, recall, f1, AUC = sess.run([accuracy, precision, recall, f1,auc],
                                         feed_dict = {X: train_X, y: train_y})
        prob_train = probability.eval(feed_dict = {X: train_X, y: train_y})
        prob_test = probability.eval(feed_dict = {X: test_X, y: test_y})
        prob_valid = probability.eval(feed_dict = {X: val_X, y: val_y})



        print('\nEvaluation  on training set')
        print('Accuracy:', acc[1])
        print('Precision:', prec[1])
        print('Recall:', recall[1])
        print('F1 score:', f1)
        print('AUC:', AUC[1])
        
      
    
    
#         save_path = saver.save(sess, "./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt")
#         save_path = saver.save(sess, "./checkpts/Later_v2_DATA_RNN_LSTM_8features.ckpt")

        
#         save_path = saver.save(sess, "./checkpts/OriginalDATA_RNN_LSTM_8features.ckpt")
#         save_path = saver.save(sess, "./checkpts/LaterDATA_RNN_LSTM_8features.ckpt")


#         save_path = saver.save(sess, "./checkpts/Original_RNN_LSTM_8features.ckpt")

#         save_path = saver.save(sess, "./checkpts/Later_RNN_LSTM_8features.ckpt")




    ## APP DATA
#         save_path = saver.save(sess, "./checkpts/Original_v2_APPDATA_RNN_LSTM_8features.ckpt")
        save_path = saver.save(sess, "./checkpts/Later_v2_APPDATA_RNN_LSTM_8features.ckpt")


    metric_out_df= pd.DataFrame(np.array([acc[1],prec[1],recall[1],f1,AUC[1],loss_fn,accu_val,best_loss_val,acc_test,loss_test,neurons,learning_rate,epochs,steps]).reshape(-1,14),columns =["accuracy","precision","recall","f1_score","auc","loss","accuracy_val","loss_val","accuracy_test","loss_test","neurons","learning_rate","n_epochs","steps"])
    return metric_out_df, prob_train, prob_test, prob_valid
    


    
    
    
    
def random_subsequence(X,y,seq_len):
    rnd  = random.randint(0,len(X)-seq_len)
    X_seq, y_seq = X[rnd:rnd+seq_len,:], y[rnd:rnd+seq_len,:]
    return X_seq, y_seq

    print(y_seq.shape)

## Odd plays train, even plays test and valid


### Generate subject files

In [168]:
dir_list = os.listdir("/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/")
dir_path ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"

subj_files_list =[]; ## list of subject_files fullfilling a criteria

dir_files = [i for i in os.listdir(dir_path) if i.startswith('sub')]

for subj_file_path in dir_files:

    file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
    mypath =file_path
    
    play_names = [i for i in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,i)) and i.startswith('app')]   
    
    if len(play_names) >= 50: ## criteria
        subj_files_list.append(subj_file_path)
        
    

In [82]:
len(subj_files_list)

37

### generate train, valid and test data

In [46]:
num_shuffles=1
for num, subj_file_path in enumerate(subj_files_list):
    print(num)
# for subj_file_path in [subj_files_list[0]]:
    
#     train_data,test_data, val_data = np.empty((0,task_df.columns.shape[0])),  np.empty((0,task_df.columns.shape[0])), np.empty((0,task_df.columns.shape[0]))
    train_data,test_data, val_data = np.empty((0,15)),  np.empty((0,15)), np.empty((0,15))

    file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
    mypath =file_path
        
    comp_task_train_df = pd.DataFrame()

    play_names = [i for i in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,i)) and i.startswith('app')]    

    for randomization_counter in range(0,num_shuffles):
            randomized_play_names= random.sample(play_names,len(play_names))
            
            for play_num, play_name in enumerate(randomized_play_names):
#         for play_num,play_name in enumerate(play_names):

                file_name = file_path + "/" + play_name
                task_df = pd.read_csv(file_name)
                task_df = add_releveant_features(task_df)

                if np.mod(play_num,2)==0: ## odd trials
                    train_data = np.append(train_data,task_df[task_df.TrialNum>1].values, axis=0)

                else:
                    test_data =  np.append(test_data, task_df[task_df.TrialNum>1].values[0:16], axis=0)
                    val_data =  np.append(val_data, task_df[task_df.TrialNum>1].values[16:], axis=0)


    train_data_df= pd.DataFrame(train_data,columns=task_df.columns)
    val_data_df = pd.DataFrame(test_data,columns=task_df.columns)
    test_data_df= pd.DataFrame(val_data,columns=task_df.columns)

#     file_path = file_path + "/OddEvenPlays/"
    file_path = file_path + "/OddEvenPlays/RandomizedPlays1"

#     os.mkdir(file_path)
    train_data_df.to_csv(file_path+"/train_data.csv")
    test_data_df.to_csv(file_path+"/test_data.csv")
    val_data_df.to_csv(file_path+"/val_data.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [169]:
def randomize_trials(df):

    locs= random.sample([a for a in range(0,df.shape[0])],df.shape[0])
    # len(locs)
    df = add_releveant_features(df.loc[locs])
    
    ### get rid of first index since it contains NaN for previous trials
    df  = df.iloc[1:]
    
    return df

### Split and format data

In [99]:
def data_split_odd_even(train_data_df,test_data_df,val_data_df):

#     train_len = 29
#     test_len = 14
#     val_len = 15

    ##----------------- UNCOMMENT BELOW
    
    if hist_flag==0: ## CURR OPTIONS ONLY
        
    
        train_X = train_data_df[['Safe','BigRisky','SmallRisky']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)

    elif hist_flag==1: ## CURR OPTIONS, PREV ACTIONS:
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
        
        
      
    elif hist_flag==2: # CURR OPTIONS, PREV OUTCOME        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
       
        
        
        
        
        
        
        
    elif hist_flag==3: ## CURR OPTIONS, PREV ACTIONS, PREV OUTCOME
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
             
        

####### Prev O + C+ R + CurrO--------------------
    elif hist_flag==4: # CURR OPTIONS, PREV ACTIONS, PREV OUTCOME, PREV OPTIONS
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
    
    
    
    
    

    
    ######## sampling 
    
    
#### - Prev RT+C+R+O + Curr O----------------------

#     train_X = task_df.loc[task_df.TrialNum>1, ['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky','PrevRT']].values
#     train_y = task_df.loc[task_df.TrialNum>1,['Choice']].values.astype(np.int32)

#     test_X = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky','PrevRT']].values
#     test_y = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Choice']].values.astype(np.int32)






    #### PRE TRAINING
#     stop = int(0.7*len(train_X))
#     print(stop)
#     train_X, test_X, val_X, train_y, test_y, val_y= train_X[:stop], train_X[stop:stop+int((len(train_X)-stop)/2)], train_X[stop+int((len(train_X)-stop)/2):],train_y[:stop], train_y[stop:stop+int((len(train_X)-stop)/2)], train_y[stop+int((len(train_X)-stop)/2):]
    
#     train_X, test_X, val_X, train_y, test_y, val_y = train_X, test_X, test_X, train_y, test_y, test_y
    ###################################################################


    print(train_X.shape)
    print(train_y.shape)
    print(val_X.shape)
    print(val_y.shape)
    print(test_X.shape)
    print(test_y.shape)

    # # center and scale
    scaler = MinMaxScaler(feature_range=(0, 1))    
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.fit_transform(test_X)
    val_X = scaler.fit_transform(val_X)


    train_X = train_X[:,None,:]
    val_X = val_X[:,None,:]
    test_X = test_X[:,None,:]


    # # one-hot encode the outputs

    onehot_encoder = OneHotEncoder()
    encode_categorical = train_y.reshape(len(train_y), 1)
    encode_categorical_test = test_y.reshape(len(test_y), 1)
    encode_categorical_val = val_y.reshape(len(val_y),1)


    train_y = onehot_encoder.fit_transform(encode_categorical).toarray()
    test_y = onehot_encoder.fit_transform(encode_categorical_test).toarray()
    val_y = onehot_encoder.fit_transform(encode_categorical_val).toarray()

    
    return train_X, train_y, test_X, test_y, val_X,val_y


### Run main training and save output

In [ ]:
neurons = 8
hist_flag=0
randomize_trials_flag=True


for num, subj_file_path in enumerate(subj_files_list[1:]):
#     print(num)
# for subj_file_path in [subj_files_list[0]]:
    
    file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
                
#     file_path = file_path + "/OddEvenPlays"
    file_path = file_path + "/OddEvenPlays/RandomizedPlays1"



    train_data_df= pd.read_csv(file_path+"/train_data.csv")
    test_data_df = pd.read_csv(file_path+"/test_data.csv")
    val_data_df = pd.read_csv(file_path+"/val_data.csv")
    

    if randomize_trials_flag==True:
        file_path = file_path + "/RandomizeTrials"
        os.mkdir(file_path)
        train_data_df = randomize_trials(train_data_df)
        val_data_df = randomize_trials(val_data_df)
        test_data_df = randomize_trials(test_data_df)


        
    
    
    train_X, train_y, test_X, test_y,val_X,val_y = data_split_odd_even(train_data_df,test_data_df,val_data_df)

    pretraining = False; 
    metric_out_df, prob_train, prob_test, prob_val = train_RNN(neurons,train_X,train_y,test_X,test_y,val_X,val_y)
    
    print(metric_out_df)
    
   
 
    prob_train_df = pd.DataFrame(prob_train.reshape(-1,2),columns = {'action_0','action_1'})
    prob_test_df = pd.DataFrame(prob_test.reshape(-1,2),columns = {'action_0','action_1'})
    prob_val_df = pd.DataFrame(prob_val.reshape(-1,2),columns = {'action_0','action_1'})

    if hist_flag==0:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currO_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currO_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currO_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currO_neurons="+str(neurons)+".csv")
    
    
    elif hist_flag==1:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currOprevC_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currOprevC_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currOprevC_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currOprevC_neurons="+str(neurons)+".csv")
        
        
    elif hist_flag==2:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currOprevR_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currOprevR_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currOprevR_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currOprevR_neurons="+str(neurons)+".csv")

    elif hist_flag==3:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currOprevRC_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currOprevRC_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currOprevRC_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currOprevRC_neurons="+str(neurons)+".csv")
# ################################
    elif hist_flag==4:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currprev_opts_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currentprevopts_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currentprevopts_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currentprevopts_neurons="+str(neurons)+".csv")
# #############################

(4262, 3)
(4262, 1)
(2351, 3)
(2351, 1)
(1910, 3)
(1910, 1)


/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.5329700708389282
Step: 100  	Training accuracy: 0.7595025897026062
Step: 100  	Valid loss: 0.5515514016151428
Step: 200  	Training loss: 0.5218982100486755
Step: 200  	Training accuracy: 0.7524442672729492
Step: 200  	Valid loss: 0.5457583069801331
Step: 300  	Training loss: 0.5161791443824768
Step: 300  	Training accuracy: 0.7510324716567993
Step: 300  	Valid loss: 0.5395402312278748
Step: 400  	Training loss: 0.5087778568267822
Step: 400  	Training accuracy: 0.7504274249076843
Step: 400  	Valid loss: 0.5313678979873657
Step: 500  	Training loss: 0.49882179498672485
Step: 500  	Training accuracy: 0.7500912547111511
Step: 500  	Valid loss: 0.5202193260192871
Step: 600  	Training loss: 0.4848722219467163
Step: 600  	Training accuracy: 0.7498773336410522
Step: 600  	Valid loss: 0.5044313669204712
Step: 700  	Training loss: 0.46652477979660034
Step: 700  	Training accuracy: 0.7497292160987854
Step: 700  	Valid loss: 0.48366957902908325
Step: 800  	Training los

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.6290060877799988
Step: 100  	Training accuracy: 0.6268260478973389
Step: 100  	Valid loss: 0.5800244808197021
Step: 200  	Training loss: 0.6019282937049866
Step: 200  	Training accuracy: 0.6505757570266724
Step: 200  	Valid loss: 0.575343132019043
Step: 300  	Training loss: 0.5909926295280457
Step: 300  	Training accuracy: 0.6494817733764648
Step: 300  	Valid loss: 0.5763649940490723
Step: 400  	Training loss: 0.5782650113105774
Step: 400  	Training accuracy: 0.6499620079994202
Step: 400  	Valid loss: 0.5667392611503601
Step: 500  	Training loss: 0.5694509744644165
Step: 500  	Training accuracy: 0.6542152762413025
Step: 500  	Valid loss: 0.5590448975563049
Step: 600  	Training loss: 0.5651047229766846
Step: 600  	Training accuracy: 0.6598212122917175
Step: 600  	Valid loss: 0.5551224946975708
Step: 700  	Training loss: 0.5630680918693542
Step: 700  	Training accuracy: 0.6652356386184692
Step: 700  	Valid loss: 0.5533484220504761
Step: 800  	Training loss: 0

...

Step: 100  	Training loss: 0.5396569967269897
Step: 100  	Training accuracy: 0.7330677509307861
Step: 100  	Valid loss: 0.5569839477539062
Step: 200  	Training loss: 0.5042269229888916
Step: 200  	Training accuracy: 0.7308210134506226
Step: 200  	Valid loss: 0.5325774550437927
Step: 300  	Training loss: 0.48480817675590515
Step: 300  	Training accuracy: 0.7303643822669983
Step: 300  	Valid loss: 0.5149262547492981
Step: 400  	Training loss: 0.46527913212776184
Step: 400  	Training accuracy: 0.7301679253578186
Step: 400  	Valid loss: 0.49529093503952026
Step: 500  	Training loss: 0.4488104581832886
Step: 500  	Training accuracy: 0.7341144680976868
Step: 500  	Valid loss: 0.4791286587715149
Step: 600  	Training loss: 0.436389297246933
Step: 600  	Training accuracy: 0.743145227432251
Step: 600  	Valid loss: 0.46692830324172974
Step: 700  	Training loss: 0.4275914430618286
Step: 700  	Training accuracy: 0.7514829635620117
Step: 700  	Valid loss: 0.4583262503147125
Step: 800  	Training loss

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.6280566453933716
Step: 100  	Training accuracy: 0.6830543875694275
Step: 100  	Valid loss: 0.6592324376106262
Step: 200  	Training loss: 0.6128659844398499
Step: 200  	Training accuracy: 0.6731775403022766
Step: 200  	Valid loss: 0.650024950504303
Step: 300  	Training loss: 0.5856335163116455
Step: 300  	Training accuracy: 0.6712013483047485
Step: 300  	Valid loss: 0.6224888563156128
Step: 400  	Training loss: 0.5132094025611877
Step: 400  	Training accuracy: 0.6796232461929321
Step: 400  	Valid loss: 0.5473513007164001
Step: 500  	Training loss: 0.4489913880825043
Step: 500  	Training accuracy: 0.6981395483016968
Step: 500  	Valid loss: 0.4774826169013977
Step: 600  	Training loss: 0.40886178612709045
Step: 600  	Training accuracy: 0.7162972092628479
Step: 600  	Valid loss: 0.43192780017852783
Step: 700  	Training loss: 0.3848098814487457
Step: 700  	Training accuracy: 0.7324907183647156
Step: 700  	Valid loss: 0.40348291397094727
Step: 800  	Training loss

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.618192732334137
Step: 100  	Training accuracy: 0.6644407510757446
Step: 100  	Valid loss: 0.6277629733085632
Step: 200  	Training loss: 0.5458860993385315
Step: 200  	Training accuracy: 0.6799628734588623
Step: 200  	Valid loss: 0.5802034735679626
Step: 300  	Training loss: 0.5050807595252991
Step: 300  	Training accuracy: 0.7023265957832336
Step: 300  	Valid loss: 0.5439063906669617
Step: 400  	Training loss: 0.46951425075531006
Step: 400  	Training accuracy: 0.7146946787834167
Step: 400  	Valid loss: 0.5082884430885315
Step: 500  	Training loss: 0.44612938165664673
Step: 500  	Training accuracy: 0.7247819900512695
Step: 500  	Valid loss: 0.48588237166404724
Step: 600  	Training loss: 0.43380552530288696
Step: 600  	Training accuracy: 0.7344904541969299
Step: 600  	Valid loss: 0.4755593240261078
Step: 700  	Training loss: 0.42698174715042114
Step: 700  	Training accuracy: 0.7424534559249878
Step: 700  	Valid loss: 0.47037020325660706
Step: 800  	Training l

In [162]:
# df_out = randomize_trials(train_data_df)

In [152]:
# train_data_df.head(10)

,Unnamed: 0,TrialNum,SideOfScreen,Safe,BigRisky,SmallRisky,SideChosen,Choice,Outcome,RT,Happiness,PrevOutcome,PrevChoice,PrevSafe,PrevBigRisky,PrevSmallRisky
0,0,2.0,1.0,-35.0,0.0,-92.0,NaN,1.0,0.0,3.660,NaN,0.0,0.0,0.0,0.0,0.0
1,1,3.0,1.0,-35.0,0.0,-59.0,NaN,1.0,-59.0,3.710,0.52,0.0,1.0,-35.0,0.0,-92.0
2,2,4.0,1.0,55.0,150.0,0.0,NaN,1.0,150.0,1.183,NaN,-59.0,1.0,-35.0,0.0,-59.0
3,3,5.0,1.0,35.0,85.0,0.0,NaN,1.0,0.0,1.367,NaN,150.0,1.0,55.0,150.0,0.0
4,4,6.0,1.0,-55.0,0.0,-105.0,NaN,1.0,-105.0,1.287,0.53,0.0,1.0,35.0,85.0,0.0
5,5,7.0,1.0,45.0,92.0,0.0,NaN,1.0,0.0,1.301,NaN,-105.0,1.0,-55.0,0.0,-105.0
6,6,8.0,1.0,55.0,59.0,0.0,NaN,0.0,55.0,1.786,NaN,0.0,1.0,45.0,92.0,0.0
7,7,9.0,1.0,35.0,105.0,0.0,NaN,1.0,0.0,1.298,0.54,55.0,0.0,55.0,59.0,0.0
8,8,10.0,1.0,-45.0,0.0,-79.0,NaN,1.0,-79.0,1.222,NaN,0.0,1.0,35.0,105.0,0.0
9,9,11.0,1.0,0.0,55.0,-72.0,NaN,1.0,55.0,1.238,0.54,-79.0,1.0,-45.0,0.0,-79.0


In [153]:
# df_out.head(10)

,Unnamed: 0,TrialNum,SideOfScreen,Safe,BigRisky,SmallRisky,SideChosen,Choice,Outcome,RT,Happiness,PrevOutcome,PrevChoice,PrevSafe,PrevBigRisky,PrevSmallRisky
548,548,28.0,1.0,35.0,68.0,0.0,NaN,1.0,68.0,2.237,0.50,-112.0,1.0,-35.0,0.0,-112.0
730,730,7.0,1.0,55.0,144.0,0.0,NaN,1.0,144.0,1.350,NaN,68.0,1.0,35.0,68.0,0.0
708,708,14.0,1.0,-35.0,0.0,-85.0,NaN,1.0,-85.0,1.421,0.52,144.0,1.0,55.0,144.0,0.0
349,349,3.0,1.0,55.0,90.0,0.0,NaN,0.0,55.0,2.075,0.50,-85.0,1.0,-35.0,0.0,-85.0
794,794,13.0,1.0,-55.0,0.0,-79.0,NaN,1.0,0.0,1.321,NaN,55.0,0.0,55.0,90.0,0.0
449,449,16.0,1.0,0.0,75.0,-54.0,NaN,1.0,75.0,1.622,NaN,0.0,1.0,-55.0,0.0,-79.0
446,446,13.0,1.0,55.0,118.0,0.0,NaN,1.0,0.0,1.471,NaN,75.0,1.0,0.0,75.0,-54.0
514,514,23.0,1.0,55.0,79.0,0.0,NaN,1.0,0.0,1.219,NaN,0.0,1.0,55.0,118.0,0.0
1093,1093,22.0,1.0,-35.0,0.0,-111.0,NaN,1.0,0.0,4.293,0.49,0.0,1.0,55.0,79.0,0.0
211,211,10.0,1.0,55.0,118.0,0.0,NaN,1.0,0.0,1.166,NaN,0.0,1.0,-35.0,0.0,-111.0
